In [1]:
import os
import sys
import cv2
import time
import math
import json
import carla
import pickle
import random
import numpy as np
from threading import Lock
from process_frame import process_frame
sys.path.append(r'/home/kaustubh/CARLA_15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
rgb_path = "Dataset/rgb_image"
seg_path = "Dataset/seg_image"
logs = "logs"
os.makedirs(rgb_path, exist_ok=True)
os.makedirs(seg_path, exist_ok=True)
os.makedirs(logs, exist_ok=True)

In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
traffic_manager = client.get_trafficmanager()
blueprint = world.get_blueprint_library()

In [4]:
# Destroying old actors i.e old sensors and vehicles
for x in world.get_actors().filter('*sensor*'):
    x.destroy()
for x in world.get_actors().filter('vehicle.volkswagen.t2_2021'):
    x.destroy()

# Blueprint of the vehicles and sensors
vehicle_bp = blueprint.find('vehicle.volkswagen.t2_2021')
camera_bp = blueprint.find('sensor.camera.rgb')
imu_bp = blueprint.find('sensor.other.imu')
gnss_bp = blueprint.find('sensor.other.gnss')

In [5]:
spawn_points = world.get_map().get_spawn_points()

In [6]:
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
image = None

vehicle_length = 4.442184
vehicle_width = 1.774566
vehicle_height = 1.987206

camera_location = carla.Location(
    x=vehicle_length / 2.9 , 
    y=0,
    z=2.2
)

# camera_rotation = carla.Rotation(pitch=-30, yaw=0, roll=0)

# camera_transform = carla.Transform(carla.Location(z=2.25, x=1.3))
camera_transform = carla.Transform(camera_location)

gnss_transform = carla.Transform(carla.Location(z=2.0))
imu_transform = carla.Transform(carla.Location(z=2.0))
gnss = None
imu = None
# vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])


In [ ]:
def euler_to_quaternion():
    pitch, roll = 0, 0
    yaw=vehicle.get_transform().rotation.yaw
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)

    return {
        "w": cr * cp * cy + sr * sp * sy,
        "x": sr * cp * cy - cr * sp * sy,
        "y": cr * sp * cy + sr * cp * sy,
        "z": cr * cp * sy - sr * sp * cy
    }

# Callbacks
def image_callback(data):
    global image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    image = array
    
def gnss_callback(event):
    global gnss
    gnss = {
        "latitude": event.latitude,
        "longitude": event.longitude
    }
    
def imu_callback(event):
    global imu
    imu = {
        "orientation": euler_to_quaternion(),
        "orientation_covariance": [0.0]*9,  
        "angular_velocity": {
            "x": event.gyroscope.x,
            "y": event.gyroscope.y,
            "z": event.gyroscope.z
        },
        "angular_velocity_covariance": [0.0]*9,  
        "linear_acceleration": {
            "x": event.accelerometer.x,
            "y": event.accelerometer.y,
            "z": event.accelerometer.z
        },
        "linear_acceleration_covariance": [0.0]*9 
    }

In [8]:
camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
gnss_sensor = world.spawn_actor(gnss_bp, gnss_transform, attach_to=vehicle)
imu_sensor = world.spawn_actor(imu_bp, imu_transform, attach_to=vehicle)

In [9]:
camera_sensor.listen(lambda data: image_callback(data))
gnss_sensor.listen(lambda data: gnss_callback(data))
imu_sensor.listen(lambda data: imu_callback(data))

In [10]:
# vehicle.set_autopilot(True)



# vehicle.set_autopilot(False)
while True:
    processed_image = process_frame(image)
    
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break

    combined_image = np.hstack((processed_image, image))

    cv2.imshow("Camera", combined_image)
    cv2.waitKey(1)

cv2.destroyAllWindows()

In [ ]:
frame = {'count': 0}
data_log = []
def save_data(image):
    timestamp = time.time()
    
    # processed_image = process_frame(image)
    img_name = f"{frame['count']:05d}.png"
    cv2.imwrite(os.path.join(rgb_path, img_name), image)
    # cv2.imwrite(os.path.join(seg_path, img_name), processed_image)

    control = vehicle.get_control()
    throttle = control.throttle
    steer = control.steer
    brake = control.brake
    
    data_log.append({
        'img': img_name,
        'steer': steer,
        'throttle': throttle,
        'brake': brake,
        'imu': imu,
        'gnss': gnss,
        'timestamp': timestamp
    })

    frame['count'] += 1
    if frame['count'] >= 10000:
        camera_sensor.stop()

In [ ]:
traffic_manager.ignore_lights_percentage(vehicle, 100.0)
traffic_manager.ignore_signs_percentage(vehicle, 100.0)

vehicle.set_autopilot(True)

for i in range(10000):
    try:
        save_data(image)
        time.sleep(0.1)
    except:
        pass
with open(os.path.join(logs, "logs.json"), "w") as f:
    json.dump(data_log, f, indent=2)
    print("Done logging")
    


In [3]:
import os
import cv2
from tqdm import tqdm
import json

rgb_dir = rgb_path
segmentation_dir = seg_path
os.makedirs(segmentation_dir, exist_ok=True)

with open(os.path.join(logs, "logs.json"), "r") as f:
    data = json.load(f)

for entry in tqdm(data):
    img_filename = entry["img"]
    # print(img_filename)
    rgb_path = os.path.join(rgb_dir, img_filename)
    seg_path = os.path.join(segmentation_dir, img_filename)

    rgb_image = cv2.imread(rgb_path)
    if rgb_image is None:
        print(f"Missing: {rgb_path}")
        continue

    mask = process_frame(rgb_image) 
    cv2.imwrite(seg_path, mask) 


100%|██████████| 10000/10000 [09:28<00:00, 17.60it/s]
